<a href="https://colab.research.google.com/github/asimds/26DECOPFAIL/blob/main/GUPSHUP_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#***Compilation of work for messages dataset shared in three csv files named as A,B & C*** 

In [1]:
#importing libraries

import pandas as pd
import numpy as np
import seaborn as sns

#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###***Loading dataset from 3 files (around 6 GB) & merged into one file as a pickle*** 

**columns like '2000198658', 'MASK', 'TRANSACTION ID', 'RETRY STATUS', 'PRINCIPAL ENTITYID' already been dropped before merging 3 csv file.**

In [2]:
df = pd.read_pickle('/content/drive/MyDrive/26Dec21/df_abc_merged')
df.shape
#(22754203, 11) ~ 2GB

(22754203, 11)

***Using library to reduce dataset size***

In [9]:
#!pip install dfreduce
#from dfreduce.core import DFReduce
#DFReduce(df, inplace=True).reduce()
#OUTPUT-
#1it [00:00, 276.60it/s]
#1it [00:00, 665.23it/s]
#100%|██████████| 11/11 [01:38<00:00,  8.92s/it]
#Memory reduced from 16899.85 MB to 2731.08 MB
#Reduced by 83.84% !!
#save that reduced file as a pickle
#df.to_pickle('/content/drive/MyDrive/26Dec21/df_abc_reduced')

In [10]:
#import pandas as pd
#df=pd.read_pickle('/content/drive/MyDrive/26Dec21/df_abc_reduced')

In [16]:
df.head(2)

,MOBILE_NUMBER,MESSAGE_ID,MESSAGE_TEXT,TIMESTAMP,STATUS,CAUSE,LATENCY(MINS),SMSTYPE,NUMBER_MESSAGES,DELIVERY_CODE,DLT_TEMPLATEID
0,919958839124,86937918634223399,<#> Your GalaxyCard OTP is 1234. Do not share ...,2021-06-05 16:03:31,Delivered,Delivered,0.0,TEXT,1,000,1107161518935073113
1,916201108256,273883812086644956,<#> Your GalaxyCard OTP is 480544. Do not shar...,2021-06-05 20:41:58,Delivered,Delivered,0.0,TEXT,1,000,1107161518935073113


In [17]:
df.columns

Index(['MOBILE_NUMBER', 'MESSAGE_ID', 'MESSAGE_TEXT', 'TIMESTAMP', 'STATUS',
       'CAUSE', 'LATENCY(MINS)', 'SMSTYPE', 'NUMBER_MESSAGES', 'DELIVERY_CODE',
       'DLT_TEMPLATEID'],
      dtype='object')

***We can use TIMESTAMP & LATENCY to create a new column as MESSAGE DELIVERED to analyse attributes related to time and message delivered such as part of the day, week, month and so on but the objective of problem statement is analyse user behaviour with respect to messages sent and their type.***

In [3]:
#dropping unnecessary columns
unnecessary = ['MESSAGE_ID','NUMBER_MESSAGES', 'DELIVERY_CODE','TIMESTAMP','LATENCY(MINS)','MESSAGE_TEXT']

df.drop(unnecessary,axis=1,inplace=True)

In [4]:
#creating function for adding message type with respect to TEMPLATE
def msg_type(row):
  if row['DLT_TEMPLATEID'] == '1107161821627156186':
    return 'REFERENCE'
  elif row['DLT_TEMPLATEID'] == '1107161785922013990':
    return 'REFERENCE'
  elif row['DLT_TEMPLATEID'] == '1107163366925685914':
    return 'REFERENCE'
  elif row['DLT_TEMPLATEID'] == '1107161518935073113':
    return 'OTP'
  elif row['DLT_TEMPLATEID'] == '1107161175141807843':
    return 'OVERDUE'  
  elif row['DLT_TEMPLATEID'] == '1107161959036451136':
    return 'OVERDUE'    
  elif row['DLT_TEMPLATEID'] == '1107162756669414535':
    return 'REFERENCE_REGIONAL'    
  elif row['DLT_TEMPLATEID'] == '1107162756643364843':
    return 'PAYMENT_LINK'    
  elif row['DLT_TEMPLATEID'] == '1107162081293342527':
    return 'CIBIL_OVERDUE'    
  elif row['DLT_TEMPLATEID'] == '1107162072741862430':
    return 'OVERDUE REGIONAL'    
  elif row['DLT_TEMPLATEID'] == '1107161513051556632':
    return 'REMINDER'    
  elif row['DLT_TEMPLATEID'] == '1107162764721508748':
    return 'OVERDUE_SETTLEMENT'
  elif row['DLT_TEMPLATEID'] == '1107161546691488861':
    return 'REFERENCE'
  elif row['DLT_TEMPLATEID'] == '1107161175183783550':
    return 'CIBIL_OVERDUE'            
  elif row['DLT_TEMPLATEID'] == '1107161546722606834':
    return 'ISSUE_RESOLVED'
  elif row['DLT_TEMPLATEID'] == '1107161182158375609':
   return 'SETTLEMENT'
  elif row['DLT_TEMPLATEID'] == '1107161520621426151':
    return 'REFERENCE'    
  else:
    return 'Undefined'

In [5]:
#adding new column to identify what type of messages sent w.r.t TEMPLATE_ID
df['MESSAGE_TYPE'] = df.apply(msg_type, axis=1)

In [6]:
#checking type of messages with their value counts
df.MESSAGE_TYPE.value_counts()

REFERENCE             21633676
OVERDUE                 461856
OTP                     374402
REFERENCE_REGIONAL       92348
PAYMENT_LINK             90844
CIBIL_OVERDUE            58638
OVERDUE REGIONAL         23443
REMINDER                  6613
OVERDUE_SETTLEMENT        6510
Undefined                 2814
ISSUE_RESOLVED            2444
SETTLEMENT                 615
Name: MESSAGE_TYPE, dtype: int64

In [7]:
#checking null values
df.isna().sum()

MOBILE_NUMBER        0
STATUS               0
CAUSE                0
SMSTYPE              0
DLT_TEMPLATEID    2803
MESSAGE_TYPE         0
dtype: int64

In [8]:
#discarding those messages type where we have undefined message type (cases where message text is itself null) 
df=df.loc[df['MESSAGE_TYPE'] != 'Undefined']

In [9]:
df.shape

(22751389, 6)

In [10]:
df.MESSAGE_TYPE.value_counts()

REFERENCE             21633676
OVERDUE                 461856
OTP                     374402
REFERENCE_REGIONAL       92348
PAYMENT_LINK             90844
CIBIL_OVERDUE            58638
OVERDUE REGIONAL         23443
REMINDER                  6613
OVERDUE_SETTLEMENT        6510
ISSUE_RESOLVED            2444
SETTLEMENT                 615
Name: MESSAGE_TYPE, dtype: int64

In [11]:
df.SMSTYPE.value_counts()

TEXT            22167129
FLASH             468469
UNICODE_TEXT      115791
Name: SMSTYPE, dtype: int64

In [12]:
df.shape

(22751389, 6)

In [13]:
#dropping different type of messages sent encoding to focus on text messages only
df = df.loc[df['SMSTYPE'] != 'UNICODE_TEXT']
df = df.loc[df['SMSTYPE'] != 'FLASH']

In [14]:
#dropping columns since we now have only one message type as text and mapped with what type of msgs are there w.r.t template shared by SABA
df.drop(['SMSTYPE','DLT_TEMPLATEID'], axis=1, inplace=True)

In [15]:
df.shape

(22167129, 4)

In [16]:
df.head()

,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE
0,919958839124,Delivered,Delivered,OTP
1,916201108256,Delivered,Delivered,OTP
2,916204839638,Delivered,Delivered,OTP
3,919911039015,Delivered,Delivered,OTP
4,916201108256,Delivered,Delivered,OTP


In [17]:
#removing country code from all mobile numbers
import numpy as np
df['MOBILE_NUMBER'] = df['MOBILE_NUMBER'].astype(str).str[2:12].astype(np.int64)

In [18]:
df.head()

,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE
0,9958839124,Delivered,Delivered,OTP
1,6201108256,Delivered,Delivered,OTP
2,6204839638,Delivered,Delivered,OTP
3,9911039015,Delivered,Delivered,OTP
4,6201108256,Delivered,Delivered,OTP


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22167129 entries, 0 to 6754377
Data columns (total 4 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   MOBILE_NUMBER  int64 
 1   STATUS         object
 2   CAUSE          object
 3   MESSAGE_TYPE   object
dtypes: int64(1), object(3)
memory usage: 845.6+ MB


In [20]:
df.columns

Index(['MOBILE_NUMBER', 'STATUS', 'CAUSE', 'MESSAGE_TYPE'], dtype='object')

In [21]:
#saving dataset
df.to_pickle('/content/drive/MyDrive/26Dec21/df_a+b+c')

In [22]:
#saving dataset
df.to_csv('/content/drive/MyDrive/26Dec21/df_a+b+c.csv')

#***To save crash start reading file from this check point***

In [2]:
#reading pickle file to save preprocessing of the above steps
import pandas as pd


In [3]:
df = pd.read_pickle('/content/drive/MyDrive/26Dec21/df_a+b+c')

###***Loading dataset from contacts to assign UserID to messages sent***

***We'll be using data from Contacts and Application to first assign appID with cooresponding contact and then from that appID we'll assign UserID.***

***Query for selecting dataset from Contacts***-
SELECT application_id, contact AS MOBILE_NUMBER, type
FROM contacts
where type = 'phone'


In [4]:
df1= pd.read_csv('/content/drive/MyDrive/26Dec21/contactscompile.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df1.head()

,application_id,mobile_number,type
0,429835,8800582915,phone
1,429835,8826291656,phone
2,429835,8826511422,phone
3,429835,8851693057,phone
4,429835,8860410611,phone


In [6]:
#unique appiId 
df1.application_id.nunique()

6931

In [7]:
#unique phone number in dataset
df1.mobile_number.nunique()

1031969

In [8]:
df1.drop('type', axis=1, inplace=True)

In [9]:
#rename columns to merge
df1.rename(columns={'mobile_number': 'MOBILE_NUMBER'}, inplace=True)

In [10]:
df_msg_plus_contact= pd.merge(df, df1, on='MOBILE_NUMBER')

In [11]:
df_msg_plus_contact.head(3)

,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE,application_id
0,7008928813,Delivered,Delivered,OTP,433977
1,7008928813,Delivered,Delivered,OTP,433977
2,7008928813,Delivered,Delivered,OTP,433977


In [12]:
df_msg_plus_contact.shape

(9389747, 5)

In [13]:
df_msg_plus_contact.application_id.nunique()

1694

***Query for selecting dataset from application***- select id as applications_id, user_id  
from applications

In [14]:
df3=pd.read_csv('/content/drive/MyDrive/user_appid.csv')

In [15]:
df3.head()

,application_id,user_id
0,576495,636562
1,267172,331820
2,244165,307649
3,513012,580355
4,185235,240951


In [16]:
df3.application_id.nunique()

610911

In [17]:
df_msg_plus_contact_plus_app= pd.merge(df_msg_plus_contact, df3, on='application_id')

In [18]:
df_msg_plus_contact_plus_app.head()

,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE,application_id,user_id
0,7008928813,Delivered,Delivered,OTP,433977,507722
1,7008928813,Delivered,Delivered,OTP,433977,507722
2,7008928813,Delivered,Delivered,OTP,433977,507722
3,7008928813,Delivered,Delivered,OTP,433977,507722
4,7008928813,Delivered,Delivered,OTP,433977,507722


In [19]:
df_msg_plus_contact_plus_app.drop('application_id', axis=1, inplace=True)

In [20]:
df_msg_plus_contact_plus_app.shape

(9389747, 5)

In [21]:
df_msg_plus_contact_plus_app.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9389747 entries, 0 to 9389746
Data columns (total 5 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   MOBILE_NUMBER  object
 1   STATUS         object
 2   CAUSE          object
 3   MESSAGE_TYPE   object
 4   user_id        int64 
dtypes: int64(1), object(4)
memory usage: 429.8+ MB


In [22]:
#swapping columns in dataframe
uid = df_msg_plus_contact_plus_app['user_id']
df_msg_plus_contact_plus_app.drop(labels=['user_id'], axis=1,inplace = True)
df_msg_plus_contact_plus_app.insert(0, 'user_id', uid)

In [23]:
df_msg_plus_contact_plus_app.head(3)

,user_id,MOBILE_NUMBER,STATUS,CAUSE,MESSAGE_TYPE
0,507722,7008928813,Delivered,Delivered,OTP
1,507722,7008928813,Delivered,Delivered,OTP
2,507722,7008928813,Delivered,Delivered,OTP


In [24]:
df_msg_plus_contact_plus_app.CAUSE = df_msg_plus_contact_plus_app.CAUSE.astype(str)
df_msg_plus_contact_plus_app.STATUS = df_msg_plus_contact_plus_app.STATUS.astype(str)

In [25]:
out = pd.pivot_table(data=df_msg_plus_contact_plus_app, 
                        index=['user_id',], 
                        columns=['MESSAGE_TYPE','STATUS'], 
                        values='CAUSE',
                        aggfunc='count').reset_index()


In [26]:
#flatenning hierarchical index
out.columns = [' '.join(col).strip() for col in out.columns.values]

In [27]:
out.head()

,user_id,CIBIL_OVERDUE Delivered,CIBIL_OVERDUE Operator Failed,ISSUE_RESOLVED Delivered,ISSUE_RESOLVED Operator Failed,OTP Delivered,OTP Not Sent to Operator,OTP Operator Failed,OTP Report Pending,OVERDUE_SETTLEMENT Delivered,OVERDUE_SETTLEMENT Operator Failed,PAYMENT_LINK Delivered,PAYMENT_LINK Operator Failed,REFERENCE Delivered,REFERENCE Not Sent to Operator,REFERENCE Operator Failed,REFERENCE Report Pending,SETTLEMENT Delivered,SETTLEMENT Operator Failed
0,27923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24.0,NaN,NaN,NaN,NaN,NaN
1,33399,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,36333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,17.0,NaN,1.0,NaN,NaN,NaN
3,36962,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,NaN,NaN,NaN
4,42867,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN


In [28]:
out.columns = out.columns.str.replace(' ', '_')

In [29]:
out.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1694 entries, 0 to 1693
Data columns (total 19 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   user_id                             1694 non-null   int64  
 1   CIBIL_OVERDUE_Delivered             69 non-null     float64
 2   CIBIL_OVERDUE_Operator_Failed       30 non-null     float64
 3   ISSUE_RESOLVED_Delivered            46 non-null     float64
 4   ISSUE_RESOLVED_Operator_Failed      385 non-null    float64
 5   OTP_Delivered                       664 non-null    float64
 6   OTP_Not_Sent_to_Operator            8 non-null      float64
 7   OTP_Operator_Failed                 32 non-null     float64
 8   OTP_Report_Pending                  4 non-null      float64
 9   OVERDUE_SETTLEMENT_Delivered        22 non-null     float64
 10  OVERDUE_SETTLEMENT_Operator_Failed  10 non-null     float64
 11  PAYMENT_LINK_Delivered              90 non-

In [30]:
out.fillna(0,inplace=True)

In [31]:
out.columns

Index(['user_id', 'CIBIL_OVERDUE_Delivered', 'CIBIL_OVERDUE_Operator_Failed',
       'ISSUE_RESOLVED_Delivered', 'ISSUE_RESOLVED_Operator_Failed',
       'OTP_Delivered', 'OTP_Not_Sent_to_Operator', 'OTP_Operator_Failed',
       'OTP_Report_Pending', 'OVERDUE_SETTLEMENT_Delivered',
       'OVERDUE_SETTLEMENT_Operator_Failed', 'PAYMENT_LINK_Delivered',
       'PAYMENT_LINK_Operator_Failed', 'REFERENCE_Delivered',
       'REFERENCE_Not_Sent_to_Operator', 'REFERENCE_Operator_Failed',
       'REFERENCE_Report_Pending', 'SETTLEMENT_Delivered',
       'SETTLEMENT_Operator_Failed'],
      dtype='object')

In [33]:
import numpy as np

msg_smm = out.groupby('user_id')['CIBIL_OVERDUE_Delivered', 'CIBIL_OVERDUE_Operator_Failed',
       'ISSUE_RESOLVED_Delivered', 'ISSUE_RESOLVED_Operator_Failed',
       'OTP_Delivered', 'OTP_Not_Sent_to_Operator', 'OTP_Operator_Failed',
       'OTP_Report_Pending', 'OVERDUE_SETTLEMENT_Delivered',
       'OVERDUE_SETTLEMENT_Operator_Failed', 'PAYMENT_LINK_Delivered',
       'PAYMENT_LINK_Operator_Failed', 'REFERENCE_Delivered',
       'REFERENCE_Not_Sent_to_Operator', 'REFERENCE_Operator_Failed',
       'REFERENCE_Report_Pending', 'SETTLEMENT_Delivered',
       'SETTLEMENT_Operator_Failed'].agg([np.sum,np.mean, np.max, np.min]).reset_index()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [34]:
msg_smm.head()

user_id CIBIL_OVERDUE_Delivered       ... SETTLEMENT_Operator_Failed          
                              sum mean  ...                       mean amax amin
0   27923                     0.0  0.0  ...                        0.0  0.0  0.0
1   33399                     0.0  0.0  ...                        0.0  0.0  0.0
2   36333                     0.0  0.0  ...                        0.0  0.0  0.0
3   36962                     0.0  0.0  ...                        0.0  0.0  0.0
4   42867                     0.0  0.0  ...                        0.0  0.0  0.0

[5 rows x 73 columns]

In [35]:
#flatenning hierarchical index
msg_smm.columns = [' '.join(col).strip() for col in msg_smm.columns.values]

In [36]:
msg_smm.head()

,user_id,CIBIL_OVERDUE_Delivered sum,CIBIL_OVERDUE_Delivered mean,CIBIL_OVERDUE_Delivered amax,CIBIL_OVERDUE_Delivered amin,CIBIL_OVERDUE_Operator_Failed sum,CIBIL_OVERDUE_Operator_Failed mean,CIBIL_OVERDUE_Operator_Failed amax,CIBIL_OVERDUE_Operator_Failed amin,ISSUE_RESOLVED_Delivered sum,ISSUE_RESOLVED_Delivered mean,ISSUE_RESOLVED_Delivered amax,ISSUE_RESOLVED_Delivered amin,ISSUE_RESOLVED_Operator_Failed sum,ISSUE_RESOLVED_Operator_Failed mean,ISSUE_RESOLVED_Operator_Failed amax,ISSUE_RESOLVED_Operator_Failed amin,OTP_Delivered sum,OTP_Delivered mean,OTP_Delivered amax,OTP_Delivered amin,OTP_Not_Sent_to_Operator sum,OTP_Not_Sent_to_Operator mean,OTP_Not_Sent_to_Operator amax,OTP_Not_Sent_to_Operator amin,OTP_Operator_Failed sum,OTP_Operator_Failed mean,OTP_Operator_Failed amax,OTP_Operator_Failed amin,OTP_Report_Pending sum,OTP_Report_Pending mean,OTP_Report_Pending amax,OTP_Report_Pending amin,OVERDUE_SETTLEMENT_Delivered sum,OVERDUE_SETTLEMENT_Delivered mean,OVERDUE_SETTLEMENT_Delivered amax,OVERDUE_SETTLEMENT_Delivered amin,OVERDUE_SETTLEMENT_Operator_Failed sum,OVERDUE_SETTLEMENT_Operator_Failed mean,OVERDUE_SETTLEMENT_Operator_Failed amax,OVERDUE_SETTLEMENT_Operator_Failed amin,PAYMENT_LINK_Delivered sum,PAYMENT_LINK_Delivered mean,PAYMENT_LINK_Delivered amax,PAYMENT_LINK_Delivered amin,PAYMENT_LINK_Operator_Failed sum,PAYMENT_LINK_Operator_Failed mean,PAYMENT_LINK_Operator_Failed amax,PAYMENT_LINK_Operator_Failed amin,REFERENCE_Delivered sum,REFERENCE_Delivered mean,REFERENCE_Delivered amax,REFERENCE_Delivered amin,REFERENCE_Not_Sent_to_Operator sum,REFERENCE_Not_Sent_to_Operator mean,REFERENCE_Not_Sent_to_Operator amax,REFERENCE_Not_Sent_to_Operator amin,REFERENCE_Operator_Failed sum,REFERENCE_Operator_Failed mean,REFERENCE_Operator_Failed amax,REFERENCE_Operator_Failed amin,REFERENCE_Report_Pending sum,REFERENCE_Report_Pending mean,REFERENCE_Report_Pending amax,REFERENCE_Report_Pending amin,SETTLEMENT_Delivered sum,SETTLEMENT_Delivered mean,SETTLEMENT_Delivered amax,SETTLEMENT_Delivered amin,SETTLEMENT_Operator_Failed sum,SETTLEMENT_Operator_Failed mean,SETTLEMENT_Operator_Failed amax,SETTLEMENT_Operator_Failed amin
0,27923,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24.0,24.0,24.0,24.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,33399,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,36333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.0,17.0,17.0,17.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,36962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,13.0,13.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,42867,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
msg_smm.to_pickle('/content/drive/MyDrive/26Dec21/gupshup_message_summary')

###***Till now the work done is based on the CSV file shared in three part and analysing with respect to user to quantify the type of messages sent. This file can be now be merged with each users email and payment traits(feature)***